In [1]:
from datasets import load_dataset

ds = load_dataset("Biddls/Onion_News")
ds['train'][0]

{'text': 'Relaxed Marie Kondo Now Says She Perfectly Happy Living In Waist-High Sewage #~# LOS ANGELES—Admitting that she’d made some major lifestyle changes since developing her famous KonMari method, a relaxed Marie Kondo told reporters Tuesday that she was now perfectly happy living in waist-high sewage. “The truth is, while I used to be very hard on myself about keeping everything clean, I’m now able to find peace living my life half-submerged in a large, fetid pool of human waste,” said Kondo, who added that while things like tidiness, organization, and minimalism used to spark joy for her, she now felt that same warmth from wading, floating, and swimming in the many gallons of untreated urine and feces that currently filled her home. “It was difficult, but once I had my children, I began to find it impossible to remove, clean, and sanitize the unending stream of excrement that bubbled up out of my toilets and filled my house to the point of collapse. While I used to hate it, I no

In [2]:
SEP = '#~#'

def get_headline(text):
    return text.split(SEP)[0].strip()

headlines = [get_headline(ds['train'][i]['text']) for i in range(20)]
headlines

['Relaxed Marie Kondo Now Says She Perfectly Happy Living In Waist-High Sewage',
 'U.S. Officials Call For Correct Amount Of Violence',
 'Kamala Harris Asks Communications Assistant If She Can Take Them Out For Coffee And Pick Their Brain Sometime',
 '25 Arrested In Fake Nursing School Diploma Scheme',
 'World’s Oldest American Dies At 72',
 'Report: Everyone Laughing At What Is A Very Silly Misunderstanding, But Don’t Be Fooled—Even Now, The Seeds Of Resentment Are Taking Root',
 'CEOs Explain How They Will Use ChatGPT',
 'FDA Moves To Ease Blood Donation Rules For Gay And Bisexual Men',
 'Study Shows Humans Still Have Genes To Grow Full Coat Of Body Hair',
 'Look What Happens When You Leave A McDonald’s Hamburger Out On A Counter For A Year',
 'Biden Secures Nation Extra Trash Can',
 'ChatGPT Forced To Take Bar Exam Even Though Dream Was To Be AI Art Bot',
 'Man Has Watched All 761 Movies',
 'David Cronenberg Once Again Leaves Doctor’s Appointment Disappointed By Lack Of Body Horror'

In [3]:
import yake

kw_extractor = yake.KeywordExtractor(lan='en', n=3, top=2)

for headline in headlines[:5]:
    print(headline)
    print(kw_extractor.extract_keywords(headline))
    print()

Relaxed Marie Kondo Now Says She Perfectly Happy Living In Waist-High Sewage
[('Perfectly Happy Living', np.float64(0.0032173869679631944)), ('Relaxed Marie Kondo', np.float64(0.0035308295728302113))]

U.S. Officials Call For Correct Amount Of Violence
[('Officials Call', np.float64(0.012602360123953448)), ('Amount Of Violence', np.float64(0.012602360123953448))]

Kamala Harris Asks Communications Assistant If She Can Take Them Out For Coffee And Pick Their Brain Sometime
[('Harris Asks Communications', np.float64(0.02140921543860024)), ('Communications Assistant', np.float64(0.02140921543860024))]

25 Arrested In Fake Nursing School Diploma Scheme
[('School Diploma Scheme', np.float64(0.001881309737406442)), ('Fake Nursing School', np.float64(0.0032173869679631944))]

World’s Oldest American Dies At 72
[('Oldest American Dies', np.float64(0.0032173869679631944)), ('Oldest American', np.float64(0.02140921543860024))]



In [8]:
import spacy


nlp = spacy.load('en_core_web_sm')

def extract_topic(text):
    doc = nlp(text.lower()) # lower casing the text seems to result in better parsing
    entities = doc.ents
    for entity in entities:
        if entity.label_ == 'PERSON':
            return text[entity.start_char:entity.end_char]

count = 0
for text in ds['train']['text']:
    headline = get_headline(text)
    topic = extract_topic(headline)
    if topic:
        count += 1
print(count)

5906
